In [1]:
import urllib.request
import lxml.html
import datetime
import re

In [2]:
#To update data, run these next 2 cells STARTING HERE
#then run the 4 cells on "Plotting HASP downlinked info"

first_half_of_path = 'https://laspace.lsu.edu/hasp/groups/2021/data/'
#a list of all the links that we need to get files/data from (we add to this)
data_file_links = []
#find links on this site
connection = urllib.request.urlopen('https://laspace.lsu.edu/hasp/groups/2021/data/data.php?pname=Payload_03&py=2021')
dom = lxml.html.fromstring(connection.read())

#idk what this next line does exactly, but it gives all the links on the webpage
for link in dom.xpath('//a/@href'): # select the url in href for all a tags(links)
    #find all of the links that start with "Payload"; these are the ones that contain files with data
    if (link[:3] == "Pay") & ('old' not in link) & ('07-30' in link): #get rid of this 3rd condition for more files
        #add the beginning of the url to the link
        link = first_half_of_path + link
        #add link to list of data we need to parse later on
        data_file_links.append(link)
# print(data_file_links)
#need to read these files (using the links we just found) and add them together based on their timestamps!

In [3]:
#combines adjacent or same days into one big file and names each big file based on the starting day
#saves these files for later use (in a different code is probably most convenient)

previous_date = datetime.datetime(2020, 12, 31) #the date each file was recorded, starting at an arbitrary value
file_count = -1 #a count for how many big files are created

#a list containing the 'title' of each file, along with the string content
# i.e. [ [title1, content1], [title2, content2] ... ]
all_big_files = [] 
year = 2021
#change this depending on the size of the time gap between data files we want
day_gap = datetime.timedelta(days = 1)

for link in data_file_links:
    # https://laspace.lsu.edu/hasp/groups/2021/data/data.php?pname=Payload_03&py=2021
    month = int(link[61:63])
    day = int(link[64:66])
    date = datetime.datetime(year, month, day)
    print(month, day)
    
#     #conditional for just creating the July 30th files (removable)
#     if month == 30:
    if date - previous_date < day_gap:
        #write the data from this link to the current file
        with urllib.request.urlopen(link) as f:
            all_big_files[file_count][1] += f.read().decode('ascii', 'ignore')

    else:
        #start a new file, recording the current link date as a 'title',
        #so it can be used to identify the file when we write them
        with urllib.request.urlopen(link) as f:
            new_file_start = f.read().decode('ascii', 'ignore')
            all_big_files.append( [link[61:83], new_file_start] )
            print("new 'file' created")
            file_count += 1 #update this counter so that we add new data to the right file later on

    #change these variables for the next iteration of the loop
    previous_date = date

for i in all_big_files:
    #setting a file name based on the date of the first entry in the file
    new_file_name = 'HASP_downlink_' + i[0]
    new_file_name = new_file_name.replace(".raw", ".csv", 1)
    #write everything to the .csv file
        
    with open(new_file_name, 'w', encoding = 'ascii') as f:
        six_zeros_indices = [m.start() for m in re.finditer('000000', i[1])]
        print(six_zeros_indices)
        extra_spaces = 0
        for j in six_zeros_indices:
            if i[1][j - 1 + extra_spaces] != '\n':
                i[1] = i[1][:j + extra_spaces] + '\n' + i[1][j + extra_spaces:]
                extra_spaces += 1
        i[1] = i[1].replace('"', '%')
        f.write(i[1])

7 30
new 'file' created
7 30
7 30
7 30
7 30
7 30
7 30
7 30
7 30
7 30
7 30
7 30
7 30
7 30
[1, 344, 686, 1029, 1371, 1714, 2064, 2415, 2764, 3115, 3469, 3822, 4176, 4528, 4880, 5234, 5587, 5938, 6290, 6641, 6994, 7346, 7698, 8049, 8403, 8756, 9108, 9459, 9809, 10160, 10509, 10859, 11209, 11557, 11906, 12256, 12606, 12955, 13304, 13654, 14002, 14351, 14701, 15049, 15399, 15748, 16097, 16446, 16796, 17145, 17495, 17844, 18193, 18542, 18899, 19256, 19613, 19971, 20329, 20686, 21043, 21400, 21756, 22114, 22472, 22830, 23188, 23546, 23905, 24262, 24623, 24983, 25345, 25706, 26066, 26426]
